In [ ]:
# 在本机未更新至2.0.8的情况下，使用vnpy2.0.8的回测逻辑
import sys
import os
from pathlib import Path
new_version_path = Path(os.getenv('VNPY2.0.8'))
sys.path.insert(0, str(new_version_path))

import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
plt.style.use('ggplot')
pd.set_option('display.float_format', '{:.2f}'.format)

from datetime import datetime, timedelta
from typing import Optional
from utility import load_data, strip_digt, comodity_to_vt_symbol

def get_hist_vol(
    commodity: str,
    start: Optional[datetime] = None,
    end: Optional[datetime] = None,
    n: int = 20
):
    vt_symbol = comodity_to_vt_symbol(commodity, data_mode='index')
    if not end:
        end = datetime.now()
    if not start:
        start = end - timedelta(days=365 * 5)
        
    df = load_data(vt_symbol, 'd', start, end)
    df["returns"] = np.log(df["close"] / df["close"].shift(1))
    df['hist_vol'] = df['returns'].rolling(n).std() * np.sqrt(252) * 100
    df = df.dropna()
    return df

In [ ]:
df = get_hist_vol('cu')
df.head()

In [ ]:
%matplotlib notebook
plt.figure(figsize=(10, 8))

close_plot = plt.subplot(2, 1, 1)
close_plot.set_title("Close")
close_plot.plot(df["close"].index, df["close"].values)

vol_plot = plt.subplot(2, 1, 2)
vol_plot.set_title("Hist volotility")
vol_plot.plot(df["hist_vol"].index, df['hist_vol'].values)

plt.tight_layout() 